In [ ]:
using DelimitedFiles, TimerOutputs
using LinearAlgebra
using Arpack
using GenericArpack
using BenchmarkTools
using ArnoldiMethod, LinearAlgebra, LinearMaps, TimerOutputs
S = readdlm("matrixS.txt")
T = readdlm("matrixT.txt")

In [ ]:

sample_count = 10
BenchmarkTools.DEFAULT_PARAMETERS.samples = sample_count
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 60
Sys.cpu_info()[1].model

In [ ]:

const c0 = 299_792_458.   # speed of light [m/s]
const ε0 = 8.85418781e-12 # vacuum permittivity [F/m]
const μ0 = 1.256637062e-6 # vacuum permeability [H/m]
const μr = 1.0
const ε  = ε0

@benchmark k², e = eigen(S,T)

In [ ]:
#shift and invert lanchos ARPACK
#@benchmark λ, ϕ = Arpack.eigs(Array(S), Array(T), nev=36, sigma=105.0, which=:LR);


times = [@elapsed Arpack.eigs(Array(S), Array(T), nev=36, sigma=105.0, which=:LR) for _ in 1:sample_count]
println("Mean execution time: ", mean(times), " seconds")
println("Standard deviation: ", std(times), " seconds")
println("Min execution time: ", minimum(times), " seconds")
println("Max execution time: ", maximum(times), " seconds")

In [ ]:

@benchmark λ, ϕ = GenericArpack.eigs(Symmetric(S), Symmetric(T), 36)

In [ ]:



const to = TimerOutput()


struct ShiftAndInvert{TA,TB,TT}
    A_lu::TA
    B::TB
    temp::TT
    sigma::eltype(TA)  # Add the shift parameter
end

function (M::ShiftAndInvert)(y, x)
    # Apply (A - σB)⁻¹ B x
    mul!(M.temp, M.B, x)                     # temp = B * x
    mul!(y, M.B, x)                          # Update temp
    ldiv!(y, M.A_lu, M.temp)                 # y = (A - σB)⁻¹ temp
end

function construct_linear_map(A, B, σ)
    # Factorize (A - σB)
    @timeit to "nest 1" begin
    A_shifted = A - σ * B
    a = ShiftAndInvert(factorize(A_shifted), B, Vector{eltype(A)}(undef, size(A, 1)), σ)
    LinearMap{eltype(A)}(a, size(A, 1), ismutating = true)
    end
end


function compute_eigenvalues(A, B, σ; nev=9, tol=1e-14, restarts=150)
    # Use shift-and-invert linear map
    
    @timeit to "throwing" begin
    decomp, = partialschur(
        construct_linear_map(A, B, σ),
        nev = nev,
        tol = tol,
        restarts = restarts,
        which = :LM,  # Dominant eigenvalues of the transformed problem
    )
    # Recover original eigenvalues
    λs_inv, X = partialeigen(decomp)
    end
    λs = σ .+ (1 ./ λs_inv)  # Transform back the eigenvalues
    return λs, X
end

# Example usage
σ = 1760.0  # Example target
#@benchmark λs, X = compute_eigenvalues(S, T, σ, nev=9)

times = [@elapsed compute_eigenvalues(S, T, σ, nev=9) for _ in 1:sample_count]
println("Mean execution time: ", mean(times), " seconds")
println("Standard deviation: ", std(times), " seconds")
println("Min execution time: ", minimum(times), " seconds")
println("Max execution time: ", maximum(times), " seconds")
to

In [ ]:
sqrt(1760) / 2π * c0 * 1e-9 